In [3]:
import os

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = "RAG"

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [6]:
# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
!pip install langchain-google-genai

  Using cached langchain_google_genai-2.1.5-py3-none-any.whl.metadata (5.2 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-6.31.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.met

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
)


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", convert_system_message_to_human=True)

In [11]:
print(model.invoke("hi").content)

Hi there! How can I help you today?


In [12]:
class_name = "flute"
prop = "9 inches, 6 holes, black colour, made of steel"
desc = model.invoke(f"write a decription about {class_name} with properties like {prop}").content
features = model.invoke(f"make {prop} concise and clear and show it as features").content
print(f"**Description** : {desc} \n\n **Features** : \n\n* **Class** : {class_name}\n{features}")

**Description** : This sleek, 9-inch black steel flute boasts six finger holes, offering a focused and resonant tone.  Its compact size makes it ideal for travel or close-quarters playing, while the robust steel construction ensures durability and a bright, clear sound. The matte black finish provides a sophisticated aesthetic, making it a striking instrument for both beginners and experienced players. 

 **Features** : 

* **Class** : flute
* **Diameter:** 9 inches
* **Holes:** 6
* **Color:** Black
* **Material:** Steel


In [21]:
import os
import json
import csv
from pathlib import Path

DB_DIR = Path("../product_db")               # folder to hold our files
CSV_FILE = DB_DIR / "products.csv"
# JSON_FILE = DB_DIR / "products.json"

DB_DIR.mkdir(exist_ok=True)

def get_next_id(csv_path: Path) -> int:
    if not csv_path.exists():
        return 0
    with open(csv_path, newline='', encoding="utf-8") as f:
        return sum(1 for _ in f) - 1       # minus header
        

def write_product_record(class_name: str, desc: str, features: str):
    record = {
        "product_id": None,     # placeholder
        "class": class_name,
        "description": desc.strip(),
        "features": features.strip()
    }
    next_id = get_next_id(CSV_FILE)
    record["product_id"] = next_id

    # append to CSV (header handled automatically)
    write_header = not CSV_FILE.exists()
    with open(CSV_FILE, "a", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=record.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(record)


In [22]:
class_name = "tabla"
prop = "3 feet diameter, 2.5 feet tall, made of african wood"
desc = model.invoke(f"write a description about {class_name} with properties like {prop}").content
features = model.invoke(f"make {prop} concise and clear and show it as points").content
write_product_record(class_name, desc, features)

In [16]:
import pandas as pd

In [23]:
df = pd.read_csv(DB_DIR / "products.csv")
print(df["features"][1])

* **Diameter:** 3 feet
* **Height:** 2.5 feet
* **Material:** African wood


In [24]:
print(df["description"][1])

This imposing tabla, a true behemoth of its kind, boasts a diameter of three feet, its resonant body rising to a height of two and a half feet. Crafted from richly hued African wood, the instrument's surface displays the beautiful, natural grain patterns unique to its origin.  The sheer size suggests a powerful, deep tone, capable of filling even the largest spaces. The weight, undoubtedly substantial, hints at a density and resonance that would be exceptional. This isn't just a tabla; it's a statement piece, a commanding presence that speaks of both masterful craftsmanship and the raw power of the African timber from which it's born.


In [25]:
print(df["product_id"][1])

3


In [1]:
import torch

if torch.cuda.is_available():
    DEVICE="cude"
elif torch.backends.mps.is_available():
    DEVICE="mps"
else:
    DEVICE="cpu"

print(f"Using Device - {DEVICE}")

Using Device - mps


In [2]:
classes = ['drum', 'flute', 'guitar', 'tabla', 'violin']


In [ ]:
img_path = "/Users/yash/Library/IntelliShelf/dataset/images/violin/0002.jpg"   

In [4]:
import torch
from PIL import Image
from torchvision import transforms

classify_model = torch.load("../model/classify.pth", map_location=DEVICE, weights_only=False)
classify_model.eval()                            
classify_model.to(DEVICE)                     

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

                 
image = Image.open(img_path).convert("RGB")
input_tensor = preprocess(image).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    logits = classify_model(input_tensor)
    probs  = torch.softmax(logits, dim=1)

top_prob, pred_idx = probs.max(dim=1)
print("Predicted index:", pred_idx.item(), "  confidence:", top_prob.item())
class_name = classes[pred_idx.item()]
print(class_name)

Predicted index: 4   confidence: 0.999983549118042
violin


In [5]:
from ultralytics import YOLO
from pathlib import Path

# ------------------------------------------------------------------
# 1. configure paths and confidence threshold
# ------------------------------------------------------------------
MODEL_PATH   = Path("/Users/yash/Library/IntelliShelf/runs/detect/train2/weights/best.pt")   # your trained weights
DEFECT_CLASS = 0                 # index you used for “defect” in defect.yaml
CONF_THRES   = 0.25              # treat predictions above this as valid

# ------------------------------------------------------------------
# 2. load model once
# ------------------------------------------------------------------
defect_model = YOLO(MODEL_PATH)

def is_damaged(image_path: str | Path, conf_thresh: float = CONF_THRES) -> tuple[bool, float]:
    """
    Returns (damaged?, highest_defect_confidence)
    """
    results = defect_model.predict(source=str(image_path), imgsz=512, conf=conf_thresh, verbose=False)
    boxes = results[0].boxes

    # find max confidence for defect class
    max_conf = 0.0
    for b in boxes:
        if int(b.cls[0]) == DEFECT_CLASS:
            max_conf = max(max_conf, float(b.conf[0]))

    damaged = max_conf >= conf_thresh
    return damaged, max_conf

# ------------------------------------------------------------------
# 3. test it
# ------------------------------------------------------------------
damaged, conf = is_damaged(img_path)

print(f"Image: {img_path}")
if damaged:
    print(f"🚨 Defected (confidence {conf:.2f}).\nPlease upload another image.")
else:
    print(f"✅ Clean (no defect ≥ {CONF_THRES}).\nNext you have to write properties (features) of your product.")


Image: /Users/yash/Library/IntelliShelf/dataset/images/violin/0002.jpg
✅ Clean (no defect ≥ 0.25).
Next you have to write properties (features) of your product.


In [12]:
import os
import json
import csv
from pathlib import Path

DB_DIR = Path("../product_db")               # folder to hold our files
CSV_FILE = DB_DIR / "products.csv"
# JSON_FILE = DB_DIR / "products.json"

DB_DIR.mkdir(exist_ok=True)

def get_next_id(csv_path: Path) -> int:
    if not csv_path.exists():
        return 0
    with open(csv_path, newline='', encoding="utf-8") as f:
        return sum(1 for _ in f) - 1       # minus header
        

def write_product_record(class_name: str, desc: str, features: str, img_path: str):
    record = {
        "product_id": None,     # placeholder
        "class": class_name,
        "description": desc.strip(),
        "features": features.strip(),
        "image_path": img_path
    }
    next_id = get_next_id(CSV_FILE)
    record["product_id"] = next_id

    # append to CSV (header handled automatically)
    write_header = not CSV_FILE.exists()
    with open(CSV_FILE, "a", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=record.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(record)


In [13]:
prop = "3 feet diameter, 2.5 feet tall, made of african wood"
desc = model.invoke(f"write a description about {class_name} with properties like {prop}").content
features = model.invoke(f"make {prop} concise and clear and show it as points").content
write_product_record(class_name, desc, features, img_path)

/Users/yash/Library/IntelliShelf/IntelliShelf/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [15]:
import pandas as pd 

df = pd.read_csv("../product_db/products.csv")
df

,product_id,class,description,features,image_path
0,0,violin,"This isn't a violin; it's a colossal, imposing...",* Diameter: 3 feet\n* Height: 2.5 feet\n* Mate...,/Users/yash/Library/IntelliShelf/dataset/image...


In [19]:
print(df["image_path"][0])

/Users/yash/Library/IntelliShelf/dataset/images/violin/0002.jpg


In [18]:
print(df["features"][0])

* Diameter: 3 feet
* Height: 2.5 feet
* Material: African wood
